In [1]:
import os
os.environ["OTEL_SDK_DISABLED"] = "true"

In [2]:
import sys
print(sys.getrecursionlimit())

3000


In [4]:
import sys
sys.setrecursionlimit(10**6)

In [5]:
from preauth_provider_side_crew.src.preauth_provider_side_crew.output_models.pydantic_output_models import PatientInfoModel

ModuleNotFoundError: No module named 'preauth_provider_side_crew'

In [ ]:
# crewai_preauth_provider.py
from crewai import Agent, Task, Crew, Process
from crewai import LLM
from crewai.tools import tool
from typing import Any, Dict

# ---------- TOOL SIMULATING INSURER SUBMISSION ----------
@tool
def submit_preauth_to_insurer(preauth_data: dict) -> str:
    """
    Simulates submission of a pre-auth form to the insurer's portal.
    """
    return f"✅ Preauth submitted successfully. Reference ID: {hash(str(preauth_data)) % 100000}"

# ---------- LLM SETUP ----------
llm = LLM(model="ollama/deepseek-r1:1.5b", base_url="http://localhost:11434", timeout=6000)

# ---------- AGENTS ----------


patient_intake_agent = Agent(
    role="Patient Intake Specialist",
    goal="Accurately capture patient demographics and insurance card details for registration.",
    backstory="You are responsible for ensuring all patient demographics and insurance details are digitized and structured \
               correctly in EMR-compatible format before moving forward in the workflow.",
    verbose=True,
    llm=llm,
)

gather_clinical_notes_agent = Agent(
    role="Clinical Documentation Specialist",
    goal="Extract and structure clinical information from physician notes using NLP.",
    backstory="You specialize in processing physician documentation to identify and organize diagnoses, procedures, \
               comorbidities, and treatment recommendations in structured format.",
    verbose=True,
    llm=llm,
)

generate_standard_codes_agent = Agent(
    role="Medical Coding Specialist",
    goal="Translate extracted clinical concepts into standardized ICD-10, CPT/HCPCS, and DRG codes.",
    backstory="You are trained in medical coding standards and ensure accurate mapping of diagnoses and procedures to \
               billing-compliant codes using official references.",
    verbose=True,
    llm=llm,
)

estimate_treatment_costs_agent = Agent(
    role="Cost Estimation Analyst",
    goal="Estimate treatment costs using hospital pricing data and structured medical inputs.",
    backstory="You analyze medical procedures, consumables, and length of stay to create accurate, itemized cost \
               estimates aligned with hospital billing systems.",
    verbose=True,
    llm=llm,
)

summarize_clinical_justification_agent = Agent(
    role="Clinical Justification Writer",
    goal="Prepare guideline-compliant justification summaries for planned treatments.",
    backstory="You are an expert in evidence-based care standards such as MCG and InterQual. You ensure treatment \
               requests are justified with clear, compliant clinical reasoning.",
    verbose=True,
    llm=llm,
)

format_preauth_form_agent = Agent(
    role="Preauthorization Document Specialist",
    goal="Compile all patient, insurance, coding, cost, and justification data into payer-compliant request formats.",
    backstory="You are skilled at formatting prior-authorization requests into insurer-compliant structures such as \
               EDI 278, FHIR bundles, PDF forms, or payer web portals.",
    verbose=True,
    llm=llm,
)


# -------------------
# Tasks
# -------------------

collect_patient_info = Task(
    description="Extract patient demographics (first name, last name, date of birth, gender, and contact details) from the input. \
       Input: {patient_info} \
       Digitize both sides of the patient's insurance card and extract key details such as policy number, insurer name, and group ID. \
       Input: {insurance_card}",
    expected_output=PatientInfoModel,
    agent=patient_intake_agent
)

gather_clinical_notes = Task(
    description="Process initial physician notes to capture presenting symptoms, provisional diagnosis, and treatment recommendations. \
        Apply NLP to extract and structure key clinical concepts such as diagnoses, procedures, and comorbidities. \
        Input: {clinical_notes}",
    expected_output="Structured summary of physician notes with extracted diagnoses, procedures, and comorbidities linked to the patient record.",
    agent=gather_clinical_notes_agent
)

generate_standard_codes = Task(
    description="Map extracted clinical concepts to standardized codes (ICD-10 for diagnoses, CPT/HCPCS for procedures, DRG for grouping) \
        using the provided coding references. \
        Input: {coding_reference}",
    expected_output="Validated mapping of diagnoses and procedures to ICD-10, CPT/HCPCS, and DRG codes.",
    agent=generate_standard_codes_agent
)

estimate_treatment_costs = Task(
    description="Estimate the overall treatment cost by combining procedure details, length of stay, and consumables. \
        Use hospital pricing data to generate itemized estimates. \
        Input: {pricing_data}",
    expected_output="Structured itemized treatment cost estimate in JSON format.",
    agent=estimate_treatment_costs_agent
)

summarize_clinical_justification = Task(
    description="Create a clinical justification summary for the planned treatment, ensuring compliance with guideline frameworks \
        (e.g., MCG or InterQual). \
        Input: {clinical_notes}",
    expected_output="Guideline-compliant justification summary supporting the requested treatment.",
    agent=summarize_clinical_justification_agent
)

format_preauth_form = Task(
    description="Integrate outputs from prior tasks (patient demographics, insurance details, diagnosis/procedure codes, cost estimates, \
        and justification). Format the compiled information into payer-compliant templates (EDI 278, FHIR bundle, PDF, or web form).",
    expected_output="Complete, well-structured, and insurer-compliant preauthorization request document.",
    agent=format_preauth_form_agent
)


# -------------------
# Crew (Workflow)
# -------------------

provider_preauth_crew = Crew(
    agents=[
        # patient_intake_agent,
        gather_clinical_notes_agent,
        generate_standard_codes_agent,
        # estimate_treatment_costs_agent,
        # summarize_clinical_justification_agent,
        # format_preauth_form_agent
    ],
    tasks=[
        # collect_patient_info,
        gather_clinical_notes,
        generate_standard_codes,
        # estimate_treatment_costs,
        # summarize_clinical_justification,
        # format_preauth_form
    ],
    process=Process.sequential,  # Ensures step-by-step execution
    verbose=True,
    name="Hospital PreAuthorization Provider-Side Automation"
)

# ---------- RUN WORKFLOW ----------

if __name__ == "__main__":
    input_data: Dict[str, Any] = {
        "patient_info": {
            "first_name": "Rajesh",
            "last_name": "Kumar",
            "dob": "1980-05-12",
            "gender": "Male",
            "contact": {
                "phone": "+91-9876543210",
                "email": "rajesh.kumar@example.com"
            },
            "address": "Hyderabad, Telangana, India",
            "ehr_id": "EHR-998877",
            "physician": {
                "name": "Dr. Anjali Mehta",
                "department": "Orthopedics",
                "npi": "1234567890"
            }
        },
        "insurance_card": {
            "insurer_name": "Star Health Insurance",
            "policy_number": "STAR-HI-20394857",
            "group_number": "GRP-112233",
            "valid_from": "2024-01-01",
            "valid_to": "2025-12-31",
            "network_affiliation": "In-Network"
        },
        "clinical_notes": """
                The patient presents with chronic left knee pain for over 6 months, worsening with movement. 
                X-ray confirms osteoarthritis. Recommended Arthroscopic Knee Surgery intervention with pre/post rehab. 
                No known drug allergies. Pre-op evaluation needed.
            """
        ,
        "pricing_data": {
            "procedure_name": "Arthroscopic Knee Surgery",
            "procedure_code": "29881",
            "procedure_cost": 48000,
            "estimated_length_of_stay": "2 days",
            "pre_post_care_cost": 7500,
            "consumables": 3200,
            "total_estimate": 58700
        },
        "coding_reference": {
            "icd_codes": {
                "M17.12":"Unilateral primary osteoarthritis, left knee",
                "M17.11":"Unilateral primary osteoarthritis, right knee",
                "M17.10":"Unilateral primary osteoarthritis, unspecified knee"
                
            },
            "cpt_codes": {
                "29881": "Arthroscopy, knee, surgical; with meniscectomy (medial OR lateral including any meniscal shaving)",
                "29880": "Arthroscopy, knee, surgical; with removal of loose body or foreign body",
                "29882": "Arthroscopy, knee, surgical; with repair of meniscus (medial OR lateral)"
            }
        }
    }

    result = provider_preauth_crew.kickoff(inputs=input_data)
    print("\n=== Final Output ===")
    print(result)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (3608729322.py, line 85)